In [9]:
!pip install -U pyautogen
!pip install PyVCF
!pip install pandas
!pip install flaml[automl]
!pip install scikit-learn
!pip install scikit-allel

  Using cached PyVCF-0.6.8.tar.gz (34 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      error in PyVCF setup command: use_2to3 is invalid.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
import autogen
# Define configurations for AutoGen
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

In [10]:
from autogen.coding import LocalCommandLineCodeExecutor
llm_config = {"config_list": config_list_gpt4, "seed": 42}

# executor = LocalCommandLineCodeExecutor(
#     work_dir="./", 
# )

user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 3, 
                          "workdir": "./",
                          #"executor": executor,
                          },
   human_input_mode="NEVER",
)
coder = autogen.AssistantAgent(
    name="Coder",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. You are a helpful assistant highly skilled in evaluating the quality of a given financial visualization code by providing a score from 1 (bad) - 10 (good) while providing clear rationale...
{see full system message above}
""",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, critic], messages=[], max_round=100)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
user_proxy.initiate_chat(manager, message="""Use the 3 files in ./data directory to follow these instructions. Print the head of the files to understand how to read them using pyvcf and pandas. Don't print complete files to not overload stdout.
                         
                         
Load the Data:
Load the genotype data from the 1000 Genomes Project VCF file into a DataFrame. This data should contain the genotypes for the ancestry-informative SNPs (AISNPs).
2. Load Ancestry Information:
Load the ancestry information from the 1000genomes_ancestry.tsv file into a DataFrame. This file should contain columns for sample IDs, population, and super population categories.
3. Preprocess the Genotype Data:
Convert the genotype data into a format suitable for analysis. This may involve encoding the genotypes numerically (e.g., 0 for reference allele, 1 for heterozygous, 2 for alternate allele).
4. One-Hot Encode Genotypes:
Apply one-hot encoding to the genotype data to prepare it for dimensionality reduction. This step transforms categorical genotype data into a binary matrix.
Dimensionality Reduction with PCA:
Use Principal Component Analysis (PCA) to reduce the dimensionality of the one-hot encoded genotype data to two dimensions. This will help visualize the genetic variation in a 2D space.
6. Join with Ancestry Information:
Join the reduced dimensionality data with the ancestry information DataFrame using the sample IDs as the key. This will add the population and super population categories to the reduced data.
Plot the Data:
Create a 2D scatter plot using a plotting library like Matplotlib or Seaborn. Plot the first two principal components on the x and y axes.
Color the points based on their ancestry categories (e.g., super population) to visualize the genetic clustering of different populations.
Customize the Plot:
Add labels, a legend, and any other necessary plot customizations to make the visualization clear and informative.
9. Review and Interpret:
Review the plot to ensure it accurately represents the genetic diversity and clustering of the 1000 Genomes Project samples by ancestry.""")

User_proxy (to chat_manager):

Use the 3 files in ./data directory to follow these instructions. Print the head of the files to understand how to read them using pyvcf and pandas. Don't print complete files to not overload stdout.
                         
                         
Load the Data:
Load the genotype data from the 1000 Genomes Project VCF file into a DataFrame. This data should contain the genotypes for the ancestry-informative SNPs (AISNPs).
2. Load Ancestry Information:
Load the ancestry information from the 1000genomes_ancestry.tsv file into a DataFrame. This file should contain columns for sample IDs, population, and super population categories.
3. Preprocess the Genotype Data:
Convert the genotype data into a format suitable for analysis. This may involve encoding the genotypes numerically (e.g., 0 for reference allele, 1 for heterozygous, 2 for alternate allele).
4. One-Hot Encode Genotypes:
Apply one-hot encoding to the genotype data to prepare it for dimensionalit

NameError: name 'cerebras_AuthenticationError' is not defined

In [7]:
import pandas as pd
samples = 'ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel'
dfsamples = pd.read_csv(samples, sep='\t')
dfsamples.set_index('sample', inplace=True)
dfsamples.drop(['Unnamed: 4', 'Unnamed: 5'], inplace=True, axis=1)

In [5]:
dfsamples.to_csv('1000genomes_ancestry.tsv', sep='\t')
